## batchsize iteration 和 epoch 的关系
+ batchsize：批大小。在深度学习中，一般采用SGD训练，即每次训练在训练集中取batchsize个样本训练
+ iteration：1个iteration等于使用batchsize个样本训练一次
+ epoch：1个epoch等于使用训练集中的全部样本训练一次
+ 举个例子，训练集有1000个样本，batchsize=10，那么：训练完整个样本集需要100次iteration，1次epoch。

## 一个LR的模型

```python
import torch
import torch.nn as nn
import numpy as np
torch.__version__
data=np.loadtxt("data/german.data-numeric")

n,l=data.shape
for j in range(l-1):
    meanVal=np.mean(data[:,j])
    stdVal=np.std(data[:,j])
    data[:,j]=(data[:,j]-meanVal)/stdVal
np.random.shuffle(data)

train_data=data[:900,:l-1]
train_lab=data[:900,l-1]-1
test_data=data[900:,:l-1]
test_lab=data[900:,l-1]-1
class LR(nn.Module):
    def __init__(self):
        super(LR,self).__init__()
        self.fc=nn.Linear(24,2) # 由于24个维度已经固定了，所以这里写24
    def forward(self,x):
        out=self.fc(x)
        out=torch.sigmoid(out)
        return out

def test(pred,lab):
    t=pred.max(-1)[1]==lab
    return torch.mean(t.float())


net=LR() 
criterion=nn.CrossEntropyLoss() # 使用CrossEntropyLoss损失
optm=torch.optim.Adam(net.parameters()) # Adam优化
epochs=1000 # 训练1000次


x=torch.from_numpy(train_data).float()
y=torch.from_numpy(train_lab).long()
y_hat=net(x)



for i in range(epochs):
    # 指定模型为训练模式，计算梯度
    net.train()
    # 输入值都需要转化成torch的Tensor
    x=torch.from_numpy(train_data).float()
    y=torch.from_numpy(train_lab).long()
    y_hat=net(x)
    loss=criterion(y_hat,y) # 计算损失
    optm.zero_grad() # 前一步的损失清零
    loss.backward() # 反向传播
    optm.step() # 优化
    if (i+1)%100 ==0 : # 这里我们每100次输出相关的信息
        # 指定模型为计算模式
        net.eval()
        test_in=torch.from_numpy(test_data).float()
        test_l=torch.from_numpy(test_lab).long()
        test_out=net(test_in)
        # 使用我们的测试函数计算准确率
        accu=test(test_out,test_l)
        print("Epoch:{},Loss:{:.4f},Accuracy：{:.2f}".format(i+1,loss.item(),accu))
```        

## tranform.ToTensor()
+ DataLoader只会接收Tensor数据,因此需要tranform

## torch show图片
+ 灰色图片
```python
plt.imshow(data[0][0].detach(),cmap="gray")
plt.show()
print()
```

+ 彩色图片
```python
img = image[0] # plt.imshow()只能接受3-D Tensor，所以也要用image[0]消去batch那一维
img = img.numpy() # FloatTensor转为ndarray
img = np.transpose(img, (1,2,0)) # 把channel那一维放到最后
plt.imshow(img)
plt.show()
```

+ 批量显示图片
```python
def imshow(imgs):
    imgs = torchvision.utils.make_grid(imgs)
    npimgs = imgs.detach().numpy()
    plt.figure(figsize=(8,8))
    plt.imshow(np.transpose(npimgs, (1,2,0)), cmap='Greys_r')
    plt.xticks([])
    plt.yticks([])
    plt.show()
```
+ 更多彩色图片
```python
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
```

## 创建自己的数据集dataset

```python
class MyData(data.Dataset):
    def __init__(self):
        data = np.loadtxt('data/german.data-numeric')
        self.len = data.shape[0]
        
        self.x_data = torch.from_numpy(data[:,:24]).float()
        self.y_data =torch.from_numpy(data[:,24]-1).long()
        
        pass
    
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
        pass
    
    def __len__(self):
        return self.len
        pass
```

## 模型训练的Train方法和Test方法

```python
def test():
    net.eval()
    for x,y in test_loader:
        output = net(x) # batch * class 的tensor
        _, prediction = output.max(1)
        corretct += prediction.eq(y).sum().item()
    accuracy = corretct / test_loader.size(0)
    print("Accuracy = ", accuracy)
def train(epoch):
    net.train()
    for e in range(epoch):
        for step,(x,y) in enumerate(train_loader):
            """
            1 清除梯度
            """
            optim.zero_grad() 
            """
            2 前向传播
            """
            y_hat = net(x)
            """
            3 - 计算loss
            """
            loss = criterion(y,y_hat)
            """
            4 - 反向传播
            """
            loss.backward()
            """
            5 - 更新参数
            """
            optim.step()
            
            if step % 100 == 0:
                test()
```

```python
## Colab 下载文件
import requests
image_url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
r = requests.get(image_url) 
with open("hymenoptera_data.zip",'wb') as f:
    f.write(r.content)
    
import os
os.system("unzip hymenoptera_data.zip")
```

## 获取参数
```python
for name,i in rnn.named_parameters():
    print(name,i.size())
```